# 第6章: 機械学習
本章では，Fabio Gasparetti氏が公開しているNews Aggregator Data Setを用い，ニュース記事の見出しを「ビジネス」「科学技術」「エンターテイメント」「健康」のカテゴリに分類するタスク（カテゴリ分類）に取り組む．

## 50. データの入手・整形

News Aggregator Data Setをダウンロードし、以下の要領で学習データ（train.txt），検証データ（valid.txt），評価データ（test.txt）を作成せよ．

1. ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．
2. 情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
3. 抽出された事例をランダムに並び替える．
4. 抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．

学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

news_corpora = "NewsAggregatorDataset/newsCorpora.csv"
columns = ["ID", "TITLE", "URL", "PUBLISHER", "CATEGORY", "STORY", "HOSTNAME", "TIMESTAMP"]

df = pd.read_csv(news_corpora, delimiter="\t", header=None, index_col=0, names=columns)

target_publisher = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]
df = df[df["PUBLISHER"].isin(target_publisher)]

# https://stackoverflow.com/questions/15772009/shuffling-permutating-a-dataframe-in-pandas
df = df.reindex(np.random.permutation(df.index))

df_size = len(df)
train_size, valid_test_size = int(df_size*0.8), int(df_size*0.1)
assert df_size == train_size + valid_test_size * 2

df_train = df.iloc[:train_size].reset_index(drop=True)
df_valid = df.iloc[train_size:train_size+valid_test_size].reset_index(drop=True)
df_test = df.iloc[train_size+valid_test_size:].reset_index(drop=True)

df_train[["CATEGORY", "TITLE"]].to_csv("train.txt", index=False, sep="\t", encoding="utf-8")
df_valid[["CATEGORY", "TITLE"]].to_csv("valid.txt", index=False, sep="\t", encoding="utf-8")
df_test[["CATEGORY", "TITLE"]].to_csv("test.txt", index=False, sep="\t", encoding="utf-8")

In [2]:
df_train.shape, df_valid.shape, df_test.shape

((10672, 7), (1334, 7), (1334, 7))

## 51. 特徴量抽出

学習データ，検証データ，評価データから特徴量を抽出し，それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ． なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．

In [3]:
import spacy

nlp = spacy.load("en_core_web_sm")

publisher2int, int2publisher = {}, {}
intpublisher = []

# https://universaldependencies.org/docs/u/pos/
open_class_words = ["ADJ", "ADV", "INTJ", "NOUN", "PROPN", "VERB"]

In [4]:
# publisher をカテゴリ変数化
def makeCategoricalPublisher(dataseries):
    for publisher in dataseries:
        if publisher not in publisher2int.keys():
            int2publisher[len(publisher2int)] = publisher
            publisher2int[publisher] = len(publisher2int)
        intpublisher.append(publisher2int[publisher])
        
    return intpublisher, int2publisher, publisher2int

In [5]:
# 入力された Title の DataSeries から Vocabulary を作成
def makeVocab(dataseries, threshold=10):
    vocab2int, int2vocab = {}, {}
    vocab_count = {}
    
    for token in [token for title in dataseries for token in nlp(title.lower())]:
        if token.pos_ in open_class_words:
            if token.lemma_ not in vocab_count.keys():
                vocab_count[token.lemma_] = 1
            else:
                vocab_count[token.lemma_] += 1

    index = 0    
    for word, count in vocab_count.items():
        if count > threshold:
            vocab2int[word] = index
            int2vocab[index] = word
            index += 1

    return vocab2int, int2vocab

In [6]:
# Bag-of-Words を作る
def makeBOW(dataseries, vocab2int):
    bow = []
    
    for title in dataseries:
        current_bag = [0] * (len(vocab2int) + 1)

        for token in nlp(title.lower()):
            if token.pos_ in open_class_words:
                if token.lemma_ in vocab2int.keys():
                    current_bag[vocab2int[token.lemma_]] += 1
                else:
                    current_bag[-1] += 1

        bow.append(current_bag)
    
    return bow

In [ ]:
intpublisher, int2publisher, publisher2int = makeCategoricalPublisher(df_train["PUBLISHER"])
vocab2int, int2vocab = makeVocab(df_train["TITLE"], 15)
bow = makeBOW(df_train["TITLE"], vocab2int)